In [ ]:
"""
Dataset Parts to Database
"""

con_db = {
    'HOST' : 'localhost', 'PORT' : '5433', 'USER' : 'postgres',
    'PASSWORD' : 'admin', 'DATABASE' : 'fireloc_db'
}

db_schema = {
    "GRID" : 'grid_ref', "GRID_PK" : 'gid', 'CELLID' : 'cellid',
    "DATA" : 'datasets', 'DATA_PK' : 'did', 'SLUG' : 'slug',
    "REL"  : 'rel_grid_data', 'GRID_FK' : 'gid', "DATA_FK" : 'did'
}

main_folder = '/home/jasp/datasets'
parts_rst   = 'mdt_m888/rsurfcountour_50'
parts_shp   = None
dataslug    = 'dem888_m50'

ff_rst = '.tif'
ff_shp = '.shp'

import os
import pandas     as pd
from gasp.pyt.oss import lst_ff
from gasp.sql.fm  import q_to_obj
from gasp.sql.to  import df_to_db
from gasp.sql.i import row_num
from gasp.sql.mng.tbl import drop_table_data

# Get Files List's
lst_rst = pd.DataFrame([[
    int(f.split('.')[0].split('_')[-1]), f
] for f in lst_ff(
    os.path.join(main_folder, parts_rst),
    file_format=ff_rst, rfilename=True
)], columns=['rid', 'rst']) if parts_rst else None

lst_shp = pd.DataFrame([[
    int(f.split('.')[0].split('_')[-1]), f
] for f in lst_ff(
    os.path.join(main_folder, parts_shp),
    file_format=ff_shp, rfilename=True
)], columns=['sid', 'shp']) if parts_shp else None

if parts_rst and parts_shp:
    fdf = lst_rst.merge(lst_shp, how='outer', left_on='rid', right_on='sid')
    if fdf.rid.isnull().values.any():
        fdf.rid.fillna(fdf.sid, inplace=True)
    
    fdf.drop('sid', axis=1, inplace=True)
    
else:
    if parts_rst:
        fdf = lst_rst
    elif parts_shp:
        fdf = lst_shp
    else:
        raise ValueError('lst_rst and lst_shp are undefined at the same time!')

if parts_rst:
    fdf.rename(columns={'rid' : 'fid'}, inplace=True)
else:
    fdf.rename(columns={'sid' : 'fid'}, inplace=True)

# Get Dataset ID
fid_dt = q_to_obj(con_db, "SELECT {} FROM {} WHERE {}='{}' LIMIT 1".format(
    db_schema["DATA_PK"], db_schema["DATA"], db_schema["SLUG"],
    dataslug
)).iloc[0][db_schema["DATA_PK"]]

# Check if we have records related with this Dataset
whr = "{} = {}".format(db_schema["DATA_FK"], str(fid_dt))
nrows = row_num(con_db, db_schema["REL"], where=whr)
if nrows:
    drop_table_data(con_db, db_schema["REL"], where=whr)
    print('{} were deleted!'.format(str(nrows)))

# Get GRID Table
grid_df = q_to_obj(con_db, "SELECT {} AS {}, {} FROM {}".format(
    db_schema["GRID_PK"], db_schema["GRID_FK"],
    db_schema["CELLID"], db_schema["GRID"]
))

# Create Relation Table
rel_df = grid_df.merge(fdf, how='inner', left_on=db_schema["CELLID"], right_on='fid')
if parts_rst:
    rel_df['rst'] = parts_rst + '/' + rel_df.rst

if parts_shp:
    rel_df['shp'] = parts_shp + '/' + rel_df.shp

rel_df[db_schema["DATA_FK"]] = fid_dt
rel_df.drop([db_schema["CELLID"], 'fid'], axis=1, inplace=True)

df_to_db(con_db, rel_df, db_schema['REL'], append=True, api='psql')